Himani Parikh - 1322085

CSCI 636 - Big Data Analytics

Hands on 8.2 : Spark Streaming

***1. Install pyspark***

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=7226c8cd335de7c5521363c61a83c01532559aef8004cce04cd221745af0e8a4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import multiprocessing

cores = multiprocessing.cpu_count()
cores

2

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [4]:
import re
def parse(line):
  match = re.search("Dm=(\d+)", line)
  if match:
    val = match.group(1)
    return [int(val)]
  return []

***Import and create streaming context.***

In [5]:
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc,1)

/usr/local/lib/python3.10/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


Create Dstream of waether data

In [6]:
lines =  ssc.socketTextStream("rtd.hpwren.ucsd.edu", 12028)

Read measurement

In [7]:
vals = lines.flatMap(parse)

***create sliding of window data***

In [8]:
window = vals.window(10,5)

***Define and call analysis function:***

In [9]:
def stats(rdd):
  print(rdd.collect())
  if rdd.count() > 0:
    print("max = {}, min ={}".format(rdd.max(), rdd.min()))

In [10]:
window.foreachRDD(lambda rdd: stats(rdd))

***Start the stream processing***

In [11]:
ssc.start()
ssc.awaitTermination()

[281, 277, 272]
max = 281, min =272
[281, 277, 272, 270, 268, 284, 286, 288]
max = 288, min =268
[270, 268, 284, 286, 288, 290, 293, 298, 302, 308]
max = 308, min =268
[290, 293, 298, 302, 308, 310, 314, 300, 302, 301, 301]
max = 314, min =290
[310, 314, 300, 302, 301, 301, 301, 299, 315, 311, 325]
max = 325, min =299
[301, 299, 315, 311, 325, 341, 341, 341, 341, 341]
max = 341, min =299
[341, 341, 341, 341, 341, 341, 341, 341, 341, 341]
max = 341, min =341
[341, 341, 341, 341, 341, 148, 167, 167, 83, 38]
max = 341, min =38
[148, 167, 167, 83, 38, 38, 38, 4, 347, 329]
max = 347, min =4
[38, 38, 4, 347, 329, 330, 329, 311, 293, 277]
max = 347, min =4
[330, 329, 311, 293, 277, 282, 282, 278, 279, 283]
max = 330, min =277
[282, 282, 278, 279, 283, 286, 288, 292, 296, 298]
max = 298, min =278
[286, 288, 292, 296, 298, 298, 302, 310, 314, 314]
max = 314, min =286
[298, 302, 310, 314, 314, 316, 317, 316, 315, 313]
max = 317, min =298
[316, 317, 316, 315, 313, 310, 309, 307, 306, 301]
max = 3

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored